In [ ]:
!pip install -U lightautoml

# Import library

In [ ]:
import pandas as pd
import numpy as np

from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task

In [ ]:
df_train = pd.read_csv('../input/tabular-playground-series-jan-2022/train.csv')
df_train.head()

### Feature generation

In [ ]:
import datetime

In [ ]:
def get_weekday(date):
    year, month, day = date.split('-')
    return datetime.date(int(year), int(month), int(day)).weekday()

In [ ]:
df_train['weekday'] = df_train['date'].apply(get_weekday)
df_train.head()

In [ ]:
list_date = np.array(df_train.date.str.split('-').to_list()).astype(int)
df_train['year'] = list_date[:,0]
df_train['month'] = list_date[:,1]
df_train['day'] = list_date[:,2]
df_train.head()

### Train 

In [ ]:
TARGET = 'num_sold'
DROP = 'row_id'
N_FOLDS = 7
N_THREADS = 4
RANDOM_STATE = 47
TEST_SIZE = 0.2
TIMEOUT = 1 * 3600

In [ ]:
def smape(y_true, y_pred):
    return 1/len(y_true) * np.sum(2 * np.abs(y_pred-y_true) / (np.abs(y_true) + np.abs(y_pred))*100)

In [ ]:
task = Task('reg',
           metric = smape)

In [ ]:
roles = {
    'target': TARGET,
    'drop': DROP,  
    
}

In [ ]:
%%time 

automl = TabularAutoML(task = task, 
                               timeout = TIMEOUT,
                               cpu_limit = N_THREADS,
                               reader_params = {
                                                'n_jobs': N_THREADS,
                                                'cv': N_FOLDS
                                               },
                               general_params = {
                               'use_algos': [['lgb_tuned', 'lgb','cb_tuned', 'cb'],['lgb_tuned', 'lgb','cb_tuned', 'cb']],
                               }
                             )

In [ ]:
%%time

oof_pred = automl.fit_predict(df_train, roles = roles, verbose=1)
print('oof_pred:\n{}\nShape = {}'.format(oof_pred, oof_pred.shape))

### Predict

In [ ]:
df_test = pd.read_csv('../input/tabular-playground-series-jan-2022/test.csv')
submission = pd.read_csv('../input/tabular-playground-series-jan-2022/sample_submission.csv')

In [ ]:
df_test['weekday'] = df_test['date'].apply(get_weekday)

In [ ]:
list_test_date = np.array(df_test.date.str.split('-').to_list()).astype(int)
df_test['year'] = list_test_date[:,0]
df_test['month'] = list_test_date[:,1]
df_test['day'] = list_test_date[:,2]

In [ ]:
test_pred = automl.predict(df_test)
test_pred.data.astype(int)

In [ ]:
submission['num_sold'] = np.round(test_pred.data, 0).astype(int)
submission.head()

In [ ]:
submission.to_csv('submission.csv', index=False)